In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'ch4': 3.6e-06}
band = [9]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
  dv = 0.001
  klin = 2e-21
  molecule = ch4
  ng_adju = [0, 0]
  ng_refs = [2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.7, 0.7), (0.4, 0.6, 0.75)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312        1 -1.837453e-01                    1 -0.159193
0.000750        2 -1.824987e-01                    2 -0.158256
0.001052        3 -1.795477e-01                    3 -0.156260
0.001476        4 -1.764536e-01                    4 -0.154076
0.002070        5 -1.734255e-01                    5 -0.151935
0.002904        6 -1.705071e-01                    6 -0.149697
0.004074        7 -1.677429e-01                    7 -0.147529
0.005714        8 -1.652087e-01                    8 -0.145272
0.008015        9 -1.630265e-01                    9 -0.143129
0.011243       10 -1.613423e-01                   10 -0.140926
0.015771       11 -1.603505e-01                   11 -0.138826
0.022122       12 -1.603519e-01                   12 -0.136903
0.031031       13 -1.617642e-01                   13 -0.135131
0.043528       14 -1.649486e-01                   14 -0.133557
0.061057       15 -1.621294e-01                   15 -0.126512
0.085645       16 -1.368983e-01                   16 -0.102696
0.120136       17 -8.919378e-02                   17 -0.061250
0.168516       18 -3.086591e-02                   18 -0.008711
0.236378       19  3.811919e-02                   19  0.055653
0.331549       20  1.193851e-01                   20  0.133798
0.465100       21  2.156393e-01                   21  0.228050
0.652400       22  3.324120e-01                   22  0.339969
0.915100       23  4.354754e-01                   23  0.438518
1.283650       24  4.066999e-01                   24  0.427581
1.800600       25  2.741742e-01                   25  0.315848
2.525700       26  1.633133e-01                   26  0.199200
3.542800       27  9.376252e-02                   27  0.108748
4.969550       28  5.502746e-02                   28  0.051378
6.970850       29  3.617524e-02                   29  0.025354
9.778100       30  2.684191e-02                   30  0.018829
13.715850      31  1.989175e-02                   31  0.018025
19.239350      32  1.314947e-02                   32  0.015794
26.987250      33  8.461898e-03                   33  0.012773
37.855300      34  5.146713e-03                   34  0.008950
53.100050      35 -1.689949e-07                   35  0.002626
73.887500      36 -7.161384e-03                   36 -0.005620
97.662500      37 -1.325052e-02                   37 -0.012625
121.437500     38 -1.812828e-02                   38 -0.018785
145.212500     39 -2.511568e-02                   39 -0.027325
168.987500     40 -3.303783e-02                   40 -0.036476
192.762500     41 -3.616828e-02                   41 -0.040287
216.537500     42 -3.367931e-02                   42 -0.038143
240.312500     43 -3.097055e-02                   43 -0.035141
264.087500     44 -2.861108e-02                   44 -0.032536
287.862500     45 -2.621884e-02                   45 -0.030021
311.637500     46 -2.366161e-02                   46 -0.027471
335.412500     47 -2.088769e-02                   47 -0.024723
359.187500     48 -1.787724e-02                   48 -0.021836
382.962500     49 -1.462817e-02                   49 -0.018649
406.737500     50 -1.117444e-02                   50 -0.015227
430.512500     51 -7.612205e-03                   51 -0.011673
454.287500     52 -4.059205e-03                   52 -0.008124
478.062500     53 -5.881797e-04                   53 -0.004458
501.837500     54  2.823763e-03                   54 -0.000865
525.612500     55  6.259695e-03                   55  0.002814
549.387500     56  9.796172e-03                   56  0.006688
573.162500     57  1.347683e-02                   57  0.010787
596.937500     58  1.732946e-02                   58  0.015119
620.712500     59  2.137354e-02                   59  0.019682
644.487500     60  2.562642e-02                   60  0.024512
668.262500     61  3.011463e-02              

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -17.745514  0.000000 -17.745514 -17.729284  8.923638e-08   
0.000624    2     -17.745533  0.000005 -17.745528 -17.729299  3.601671e-06   
0.000876    3     -17.745541  0.000008 -17.745533 -17.729305  5.032576e-06   
0.001229    4     -17.745552  0.000011 -17.745541 -17.729314  7.082934e-06   
0.001723    5     -17.745566  0.000015 -17.745551 -17.729326  1.002579e-05   
0.002417    6     -17.745587  0.000021 -17.745566 -17.729343  1.424535e-05   
0.003391    7     -17.745616  0.000031 -17.745585 -17.729366  2.030005e-05   
0.004757    8     -17.745656  0.000044 -17.745612 -17.729399  2.898010e-05   
0.006672    9     -17.745712  0.000062 -17.745650 -17.729444  4.143064e-05   
0.009359    10    -17.745790  0.000089 -17.745702 -17.729507  5.926938e-05   
0.013128    11    -17.745900  0.000126 -17.745774 -17.729596  8.484133e-05   
0.018415    12    -17.746054  0.000180 -17.745874 -17.729719  1.214709e-04   
0.025830    13    -17.746270  0.000255 -17.746015 -17.729892  1.738821e-04   
0.036232    14    -17.746575  0.000361 -17.746214 -17.730134  2.488372e-04   
0.050823    15    -17.747008  0.000508 -17.746499 -17.730472  3.559921e-04   
0.071291    16    -17.747617  0.000724 -17.746893 -17.730939  5.162415e-04   
0.100000    17    -17.748437  0.001079 -17.747358 -17.731556  7.844742e-04   
0.140271    18    -17.749489  0.001705 -17.747784 -17.732328  1.264221e-03   
0.196760    19    -17.750782  0.002792 -17.747990 -17.733240  2.117931e-03   
0.275997    20    -17.752275  0.004643 -17.747632 -17.734218  3.618431e-03   
0.387100    21    -17.753795  0.007734 -17.746061 -17.735058  6.219209e-03   
0.543100    22    -17.754879  0.012802 -17.742076 -17.735289  1.066506e-02   
0.761700    23    -17.754384  0.020915 -17.733469 -17.733918  1.809833e-02   
1.068500    24    -17.750507  0.032865 -17.717642 -17.729547  2.966536e-02   
1.498800    25    -17.743350  0.046438 -17.696912 -17.722305  4.422036e-02   
2.102400    26    -17.735060  0.057752 -17.677308 -17.713905  5.840691e-02   
2.949000    27    -17.726569  0.065639 -17.660930 -17.705152  6.963242e-02   
4.136600    28    -17.718257  0.070517 -17.647740 -17.696419  7.619975e-02   
5.802500    29    -17.710400  0.073519 -17.636881 -17.687984  7.790445e-02   
8.139200    30    -17.703036  0.076169 -17.626867 -17.679882  7.682186e-02   
11.417000   31    -17.696187  0.079741 -17.616445 -17.672122  7.637327e-02   
16.014700   32    -17.690531  0.084920 -17.605612 -17.665380  7.944938e-02   
22.464000   33    -17.687856  0.092290 -17.595566 -17.661437  8.757278e-02   
31.510500   34    -17.690003  0.103505 -17.586498 -17.662243  1.020681e-01   
44.200100   35    -17.699154  0.120392 -17.578762 -17.670394  1.236741e-01   
62.000000   36    -17.721855  0.143093 -17.578762 -17.693239  1.520555e-01   
85.775000   37    -17.769789  0.170858 -17.598931 -17.743664  1.866518e-01   
109.550000  38    -17.834969  0.198720 -17.636248 -17.813466  2.208931e-01   
133.325000  39    -17.915723  0.228420 -17.687303 -17.901414  2.559312e-01   
157.100000  40    -18.016123  0.258086 -17.758037 -18.011878  2.894299e-01   
180.875000  41    -18.141494  0.290412 -17.851082 -18.148740  3.235523e-01   
204.650000  42    -18.288432  0.335488 -17.952943 -18.306818  3.681577e-01   
228.425000  43    -18.447929  0.400134 -18.047795 -18.477070  4.309772e-01   
252.200000  44    -18.616708  0.481690 -18.135018 -18.655865  5.107931e-01   
275.975000  45    -18.793533  0.577937 -18.215596 -18.842064  6.053499e-01   
299.750000  46    -18.977356  0.687919 -18.289437 -19.034563  7.132915e-01   
323.525000  47    -19.167162  0.811087 -18.356075 -19.232138  8.334914e-01   
347.300000  48    -19.361953  0.947052 -18.414902 -19.433895  9.656134e-01   
371.075000  49    -19.560725  1.095476 -18.4652

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')